In [5]:
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from models import GCNN, AttGNN
from tqdm.auto import tqdm

npy_file = "../human_features/npy_file_test.npy"
processed_dir="../human_features/processed/"
def bump(g):
    return g
    # return Data.from_dict(g.__dict__)

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      # print(prot_1, prot_2)
      # print(glob.glob(prot_1), glob.glob(prot_2))
      #print(f'Second prot is {prot_2}')
      prot_1 = torch.load(glob.glob(prot_1)[0])
      #print(f'Here lies {glob.glob(prot_2)}')
      prot_2 = torch.load(glob.glob(prot_2)[0])
      prot_1 = bump(prot_1)
      prot_2 = bump(prot_2)
      prot_1.x = prot_1.x.to(torch.float32)
      prot_2.x = prot_2.x.to(torch.float32)
      return prot_1, prot_2, torch.tensor(self.label[index])

dataset = LabelledDataset(npy_file = npy_file ,processed_dir= processed_dir)
print(f'len test dataset: {len(dataset)}')

len test dataset: 5922


In [6]:
testloader = DataLoader_n(dataset=dataset, batch_size=4, num_workers=0)
len(testloader)

1481

In [ ]:
## test 

In [7]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
model = GCNN()
model.load_state_dict(torch.load("../human_features/GCN_50.pth")) 
model.to(device)
model.eval()
predictions = torch.Tensor()
labels = torch.Tensor()
with torch.no_grad():
    for prot_1, prot_2, label in tqdm(testloader):
      prot_1 = prot_1.to(device)
      prot_2 = prot_2.to(device)
      #print("H")
      #print(torch.Tensor.size(prot_1.x), torch.Tensor.size(prot_2.x))
      output = model(prot_1, prot_2)
      predictions = torch.cat((predictions, output.cpu()), 0)
      labels = torch.cat((labels, label.view(-1,1).cpu()), 0)
labels = labels.numpy().flatten()
predictions = predictions.numpy().flatten()

GCNN Loaded
GCNN(
  (pro1_conv1): GCNConv(1024, 1024)
  (pro1_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (pro2_conv1): GCNConv(1024, 1024)
  (pro2_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
AttGNN Loaded
AttGNN(
  (pro1_conv1): GATConv(1024, 128, heads=1)
  (pro1_fc1): Linear(in_features=128, out_features=128, bias=True)
  (pro2_conv1): GATConv(1024, 128, heads=1)
  (pro2_fc1): Linear(in_features=128, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): 

  0%|          | 0/1481 [00:00<?, ?it/s]

In [13]:
print("Labels unique values:", np.unique(labels))
print("Labels distribution:", np.bincount(labels.astype(int)))
print("Predictions range:", predictions.min(), predictions.max())
print("Predictions distribution:")
print(np.histogram(predictions, bins=10))

Labels unique values: [0.       0.254085 0.312347 0.335467 0.402325 0.448666 0.546693 0.555837
 0.564025 0.590789 0.599047 0.649687 0.657212 0.688877 0.731766 0.781149
 0.803914 0.841009 0.841369 0.84589  0.850172 0.858674 0.863618 0.890524
 0.893722 0.903702 0.922873 0.929143 0.936921 0.93774  0.946618 0.95106
 0.9573   0.958786 0.962619 0.968646 0.977758 0.982325 0.984882 0.988004
 0.988808 0.990568 0.993616]
Labels distribution: [5922]
Predictions range: 5.7578063e-07 0.6520468
Predictions distribution:
(array([  88,   29,   39,   51,  550, 3471,  840,  641,  189,   24]), array([5.7578063e-07, 6.5205202e-02, 1.3040982e-01, 1.9561444e-01,
       2.6081908e-01, 3.2602370e-01, 3.9122832e-01, 4.5643294e-01,
       5.2163756e-01, 5.8684218e-01, 6.5204680e-01], dtype=float32))


In [14]:
try:
    loss = get_mse(labels, predictions)
    acc = get_accuracy(labels, predictions, 0.5)
    prec = precision(labels, predictions, 0.5)
    sensitivity = sensitivity(labels, predictions,  0.5)
    specificity = specificity(labels, predictions, 0.5)
    f1 = f_score(labels, predictions, 0.5)
    mcc = mcc(labels, predictions,  0.5)
    auroc = auroc(labels, predictions)
    auprc = auprc(labels, predictions)
except Exception as e:
    print(f"Error calculating metrics: {e}")
    print("Detailed diagnostic information:")
    print("Labels:", labels)
    print("Predictions:", predictions)

Error calculating metrics: division by zero
Detailed diagnostic information:
Labels: [0.335467 0.335467 0.335467 ... 0.335467 0.335467 0.335467]
Predictions: [0.52831584 0.36795896 0.36130196 ... 0.38861328 0.35173368 0.37437233]


In [20]:
## convert continue valued metrics to binary metrics 
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix

def choose_optimal_threshold(actual, predicted):
    """
    Choose the optimal threshold that maximizes the F1 score
    
    Args:
        actual (np.array): True labels
        predicted (np.array): Predicted probabilities
    
    Returns:
        float: Optimal threshold
    """
    thresholds = np.linspace(0, 1, 100)
    f1_scores = []
    
    for threshold in thresholds:
        binary_actual = (actual >= np.median(actual)).astype(int)
        binary_pred = (predicted >= threshold).astype(int)
        
        # Compute confusion matrix
        tn, fp, fn, tp = confusion_matrix(binary_actual, binary_pred).ravel()
        
        # Compute F1 score
        if tp + fp == 0 or tp + fn == 0:
            f1 = 0
        else:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        
        f1_scores.append(f1)
    
    # Return threshold with max F1 score
    return thresholds[np.argmax(f1_scores)]

def get_binary_metrics(actual, predicted, threshold=0.5):
    """
    Compute binary classification metrics for continuous labels
    
    Args:
        actual (np.array): True labels
        predicted (np.array): Predicted probabilities
        threshold (float, optional): Classification threshold. If None, optimal threshold is computed.
    
    Returns:
        dict: Metrics including accuracy, precision, recall, F1, etc.
    """
    # If no threshold provided, find optimal threshold
    if threshold is None:
        threshold = choose_optimal_threshold(actual, predicted)
    
    # Convert to binary classification
    binary_actual = (actual >= np.median(actual)).astype(int)
    binary_pred = (predicted >= threshold).astype(int)
    
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(binary_actual, binary_pred).ravel()
    
    # Compute metrics
    metrics = {
        'threshold': threshold,
        'accuracy': (tp + tn) / (tp + tn + fp + fn),
        'precision': tp / (tp + fp) if tp + fp > 0 else 0,
        'recall': tp / (tp + fn) if tp + fn > 0 else 0,
        'specificity': tn / (tn + fp) if tn + fp > 0 else 0,
        'f1_score': 2 * tp / (2 * tp + fp + fn) if tp > 0 else 0,
        'auroc': roc_auc_score(binary_actual, predicted),
        'auprc': average_precision_score(binary_actual, predicted)
    }
    
    return metrics

# Modify existing metrics to use the new approach
def get_mse(actual, predicted):
    return ((actual - predicted) ** 2).mean()

def get_accuracy(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    return metrics['accuracy'] * 100.0

def precision(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    return metrics['precision']

def sensitivity(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    return metrics['recall']

def specificity(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    return metrics['specificity']

def f_score(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    return metrics['f1_score']

def mcc(actual, predicted, threshold=None):
    metrics = get_binary_metrics(actual, predicted, threshold)
    
    # Matthews Correlation Coefficient calculation
    tp = metrics['precision'] * len(actual)
    tn = metrics['specificity'] * len(actual)
    fp = tp / metrics['precision'] - tp
    fn = tp / metrics['recall'] - tp
    
    numerator = (tp * tn) - (fp * fn)
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return numerator / denominator if denominator != 0 else 0

def auroc(actual, predicted):
    return get_binary_metrics(actual, predicted)['auroc']

def auprc(actual, predicted):
    return get_binary_metrics(actual, predicted)['auprc']

In [19]:
loss = get_mse(labels, predictions)
acc = get_accuracy(labels, predictions, 0.5)
prec = precision(labels, predictions, 0.5)
sensitivity = sensitivity(labels, predictions,  0.5)
specificity = specificity(labels, predictions, 0.5)
f1 = f_score(labels, predictions, 0.5)
mcc = mcc(labels, predictions,  0.5)
auroc = auroc(labels, predictions)
auprc = auprc(labels, predictions)


print(f'loss : {loss}')
print(f'Accuracy : {acc}')
print(f'precision: {prec}')
print(f'Sensititvity :{sensitivity}')
print(f'specificity : {specificity}')
print(f'f-score : {f1}')
print(f'MCC : {mcc}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')

loss : 0.030255012972890634
Accuracy : 25.34616683552854
precision: 0.8567493112947658
Sensititvity :0.06645299145299145
specificity : 0.9581320450885669
f-score : 0.12333928217330954
MCC : -0.066497187510526
AUROC: 0.6020689678902239
AUPRC: 0.830453907547192


In [21]:
## continue-valued metrics 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def get_rmse(actual, predicted):
    """
    Root Mean Squared Error
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
    
    Returns:
        float: Root Mean Squared Error
    """
    return np.sqrt(mean_squared_error(actual, predicted))

def get_mae(actual, predicted):
    """
    Mean Absolute Error
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
    
    Returns:
        float: Mean Absolute Error
    """
    return mean_absolute_error(actual, predicted)

def get_r2_score(actual, predicted):
    """
    R-squared (Coefficient of Determination)
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
    
    Returns:
        float: R-squared score
    """
    return r2_score(actual, predicted)

def pearson_correlation(actual, predicted):
    """
    Pearson Correlation Coefficient
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
    
    Returns:
        float: Pearson correlation coefficient
    """
    return np.corrcoef(actual, predicted)[0, 1]

def spearman_correlation(actual, predicted):
    """
    Spearman Rank Correlation Coefficient
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
    
    Returns:
        float: Spearman correlation coefficient
    """
    from scipy import stats
    return stats.spearmanr(actual, predicted)[0]

# Optional: Plotting function to visualize predictions vs actual values
def plot_prediction_vs_actual(actual, predicted, title='Predictions vs Actual'):
    """
    Create a scatter plot of predictions vs actual values
    
    Args:
        actual (np.array): True values
        predicted (np.array): Predicted values
        title (str, optional): Plot title
    
    Returns:
        matplotlib.figure.Figure: Matplotlib figure
    """
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    plt.scatter(actual, predicted, alpha=0.5)
    plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(title)
    plt.tight_layout()
    
    return plt.gcf()

In [22]:
# Existing code remains the same
rmse = get_rmse(labels, predictions)
mae = get_mae(labels, predictions)
r2 = get_r2_score(labels, predictions)
pearson = pearson_correlation(labels, predictions)
spearman = spearman_correlation(labels, predictions)

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R-squared: {r2}')
print(f'Pearson Correlation: {pearson}')
print(f'Spearman Correlation: {spearman}')

# Optional: Create a visualization
fig = plot_prediction_vs_actual(labels, predictions)
fig.savefig('predictions_vs_actual_5epochs.png')
plt.close(fig)

RMSE: 0.17393968199606044
MAE: 0.11947577226948514
R-squared: 0.07805297426553204
Pearson Correlation: 0.31022231981260273
Spearman Correlation: 0.23707677788774872


In [25]:
mse = get_mse(labels, predictions)
mse

0.030255012972890634

In [26]:
# Add MSE diagnostic information
print("\nMSE Diagnostic Information:")
print(f"Labels mean: {np.mean(labels)}")
print(f"Labels standard deviation: {np.std(labels)}")
print(f"Predictions mean: {np.mean(predictions)}")
print(f"Predictions standard deviation: {np.std(predictions)}")

# Compute baseline MSE (using mean prediction)
baseline_mse = np.mean((labels - np.mean(labels))**2)
print(f"Baseline MSE (predicting mean): {baseline_mse}")
print(f"Current Model MSE: {loss}")
print(f"Improvement over baseline: {(baseline_mse - loss) / baseline_mse * 100:.2f}%")

# Visualize prediction errors
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(labels, predictions, alpha=0.5)
plt.plot([labels.min(), labels.max()], [labels.min(), labels.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Prediction Accuracy')
plt.savefig('prediction_accuracy.png')
plt.close()

# Error distribution
errors = labels - predictions
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.xlabel('Prediction Errors')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.savefig('error_distribution.png')
plt.close()


MSE Diagnostic Information:
Labels mean: 0.3667525682201959
Labels standard deviation: 0.18115306563235456
Predictions mean: 0.3716069459915161
Predictions standard deviation: 0.08013929426670074
Baseline MSE (predicting mean): 0.032816433188000156
Current Model MSE: 0.030255012972890634
Improvement over baseline: 7.81%
